In [15]:
using Colors
using AxisArrays
using Plots; plotly()

Plots.PlotlyBackend()

In [2]:
cmap = colormap("RdBu", 101)
function colorize(value)
    isnan(value) ? colorant"white" : cmap[round(Int, value * 50 + 51)]
end

colorize (generic function with 1 method)

In [3]:
include("main.jl")

c

In [4]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)

state = c.MPC.State(0.5, -2., 0.5)
time = Axis{:time}(0:0.1:3)
side = Axis{:side}([:left, :right])
result = c.MPC.run_opt(double_integrator, state, time, side);

q = getvariable(result.model, :q)
qlimb = getvariable(result.model, :qlimb)

plt = plot(time.val, getvalue(q)) 
plot!(plt, time.val, getvalue(qlimb))
plt

In [5]:
m = 1.
l = 1.
g = 10.
A = [0. 1.;
    g/l 0.]
B = [0. 1/(m*l^2.)]'
lipm = c.MPC.discretize(c.MPC.CTLinearSytstem(A, B), 0.1)

state = c.MPC.State(0.5, -2., 0.5)
time = Axis{:time}(0:0.1:3)
side = Axis{:side}([:left, :right])
result = c.MPC.run_opt(lipm, state, time, side);

q = getvariable(result.model, :q)
qlimb = getvariable(result.model, :qlimb)

plt = plot(time.val, getvalue(q)) 
plot!(plt, time.val, getvalue(qlimb))
plt

In [6]:
samples_di = c.collect_data(double_integrator, 500)
corr_di = c.correlate(samples_di);

In [7]:
samples_lipm = c.collect_data(lipm, 500)
corr_lipm = c.correlate(samples_lipm);

In [8]:
colorize.(corr_di[
    Axis{:constraint}(:force_without_contact_right),
    Axis{:contact_side}(:right)
        ])

In [9]:
colorize.(corr_lipm[
    Axis{:constraint}(:force_without_contact_right),
    Axis{:contact_side}(:right)
        ])

In [12]:
hcat(colorize.(corr_di[
    Axis{:constraint}(:normal_force_right),
    Axis{:contact_side}(:right)
        ]),
colorize.(corr_di[
    Axis{:constraint}(:normal_force_right),
    Axis{:contact_side}(:left)
        ]))

In [13]:
hcat(colorize.(corr_lipm[
    Axis{:constraint}(:normal_force_right),
    Axis{:contact_side}(:right)
        ]),
colorize.(corr_lipm[
    Axis{:constraint}(:normal_force_right),
    Axis{:contact_side}(:left)
        ]))